In [1]:
from tqdm import tqdm_notebook 
from konlpy.tag import * 
import MeCab
import string 
import warnings


from gensim import corpora
from gensim import models

import numpy as np
import re
import pickle
import matplotlib.pyplot as plt

%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning)

mecab = MeCab.Tagger()

C:\Users\poeun\anaconda3\envs\NLP\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
def mecab_nouns(text):
    nouns = []
    
    pattern = re.compile('.*\t[A-Z]+')
    
    temp =[tuple(pattern.match(token).group(0).split('\t')) for token in mecab.parse(text).splitlines()[:-1]]  
    for token in temp:
        if token[1] == 'NNG' or token[1] == 'NNP' or token[1] == 'NNB' or token[1] == 'NNBC' or token[1] == 'NP' or token[1] == 'NR':
            nouns.append(token[0])

    return nouns

def mecab_morphs(text):
    morphs = []
    
    pattern = re.compile('.*\t[A-Z]+')

    temp =[tuple(pattern.match(token).group(0).split('\t')) for token in mecab.parse(text).splitlines()[:-1]]  
        
    for token in temp:
        morphs.append(token[0])

    return morphs

def mecab_pos(text):
    pos = []
    pattern = re.compile('.*\t[A-Z]+')
    pos = [tuple(pattern.match(token).group(0).split('\t')) for token in mecab.parse(text).splitlines()[:-1]]
        
    return pos

In [3]:
def read_documents(input_file_name):
    
    corpus = []
    
    with open(input_file_name, 'rb') as f: 
        temp_corpus = pickle.load(f)
    
    for page in temp_corpus: 
        corpus += page 
    
    return corpus

def text_cleaning(docs):
    cleaned_docs = []
    for doc in docs:
        temp_doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
        cleaned_docs.append(temp_doc)

    return cleaned_docs

def define_stopwords(path):
    
    SW = set()
    for i in string.punctuation:
        SW.add(i)
    
    with open(path, 'r', encoding='utf-8') as f:
        for word in f:
            SW.add(word)

    return SW


def text_tokenizing(corpus, tokenizer):
    token_corpus = []
    if tokenizer == 'noun':
        for n in tqdm_notebook(range(len(corpus)), desc='Preprocessing'):
            token_text = mecab_nouns(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)

    elif tokenizer == 'morph':
        for n in tqdm_notebook(range(len(corpus)), desc='Preprocessing'):
            token_text = mecab_morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
    
    elif tokenizer == 'word':
        for n in tqdm_notebook(range(len(corpus)), desc='Preprocessing'):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)    

    return token_corpus

In [4]:
input_file_name = "./naver_news_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("f:/data/stopwords-ko.txt")
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="morph")

Preprocessing:   0%|          | 0/522 [00:00<?, ?it/s]

In [5]:
print(tokenized_text[3])

['본문', '내용', '플레이어', '플레이어', '오류', '우회', '위한', '함수', '추가', '부터', '까지', '이틀', '화상', '정상', '회의', '진행', '예정', '역내', '백신', '보급', '지원', '강화', '대응', '기금', '올해', '추가', '기여', '아세안', '정상', '대통령', '남방', '정책', '아세안', '실질', '협력', '추진', '높이', '평가', '대통령', '한반도', '완전', '비핵화', '항구', '평화', '정착', '위한', '아세안', '지지', '요청', '문재', '대통령', '청와대', '충무', '에서', '열린', '아세안', '화상', '정상', '회의', '에서', '기념', '촬영', '사진', '연합뉴스', '문재인', '대통령', '오후', '화상', '으로', '개최', '아세안', '정상', '회의', '참석', '이틀', '예정', '아세안', '관련', '정상', '회의', '일정', '시작', '면서', '한국', '아세안', '친구', '로서', '코로나', '함께', '극복', '포용', '지속', '가능', '미래', '함께', '만들', '나갈', '강조', '이날', '회의', '에서', '대통령', '아세안', '정상', '지난', '남방', '정책', '협력', '성과', '종합', '점검', '코로나', '위기', '보다', '미래', '함께', '만들', '나가', '위한', '아세안', '협력', '방향', '대해', '논의', '대통령', '모두', '발언', '에서', '한국', '아세안', '신뢰', '파트너', '로서', '아세안', '함께', '위기', '극복', '포용', '지속', '가능', '미래', '만들', '다는', '의지', '강조', '특히', '대통령', '우리', '나라', '글로벌', '백신', '허브', '으로서', '역내', '백신', '보급', '지원', '강화', '시켜', '나갈', '예정', 

In [6]:
dictionary = corpora.Dictionary(tokenized_text)
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

In [7]:
print(dictionary)

Dictionary(8550 unique tokens: ['가능', '가장', '각국', '감사', '강조']...)


In [8]:
corpus[2]

corpus[2][:5]

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 2)]

In [9]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf[2][:5]

[(0, 0.018916727108076122),
 (1, 0.020354589699820753),
 (2, 0.031224685165735507),
 (3, 0.029297937295090025),
 (4, 0.03827406104350658)]

In [10]:
model = models.ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary)

In [11]:
model.show_topic(0, 10)

[('백신', 0.021052355),
 ('접종', 0.020327598),
 ('으로', 0.020016687),
 ('에서', 0.015179143),
 ('코로나', 0.014071166),
 ('분기', 0.008485088),
 ('생산', 0.008364379),
 ('본문', 0.0074427095),
 ('내용', 0.0069811405),
 ('다고', 0.006506364)]

In [12]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 4

NUM_TOPIC_WORDS = 30

def build_doc_term_mat(documents):
    # 문서-단어 행렬 만들어주는 함수.
    print("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]

    return corpus, dictionary


def print_topic_words(model): # model = LDA된 결과 

    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")

    for topic_id in range(model.num_topics): 
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print('Topic ID: {}'.format(topic_id))

        for topic_word, prob in topic_word_probs:
            print('\t{}\t{}'.format(topic_word, prob))

        print('\n')

# document-term matrix를 만들고,
corpus, dictionary = build_doc_term_mat(tokenized_text)
# LDA를 실행.
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha='auto', eta='auto')
# 결과를 출력.
print_topic_words(model)

Building document-term matrix.

Printing topic words.

Topic ID: 0
	으로	0.01868796907365322
	백신	0.01469503529369831
	에서	0.014251402579247952
	생산	0.012739577330648899
	접종	0.0107877841219306
	코로나	0.010779405944049358
	내용	0.008161202073097229
	국내	0.007870790548622608
	분기	0.006170258391648531
	본문	0.0058930134400725365
	모더	0.005787328816950321
	도입	0.005332078319042921
	공급	0.0052858074195683
	정부	0.005242256913334131
	위한	0.004618252161890268
	추가	0.004427516367286444
	지난	0.004337997641414404
	라고	0.004306213464587927
	바이오	0.0042998106218874454
	부터	0.004288575146347284
	플레이어	0.004243641626089811
	다고	0.003929190803319216
	까지	0.003925771452486515
	회복	0.003779785707592964
	습니다	0.0037215761840343475
	삼성	0.003324238583445549
	방역	0.0032641631551086903
	상황	0.0032407816033810377
	로직스	0.003206168534234166
	확진	0.0031946473754942417


Topic ID: 1
	접종	0.02832021750509739
	코로나	0.0177763719111681
	으로	0.017198657616972923
	백신	0.015856606885790825
	에서	0.015709973871707916
	플레이어	0.008996620774269104
	본문	0.0081833

In [13]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim
# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행.
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data # print X 그냥 실행

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.003183  0.017341       1        1  32.248292
3      0.017487  0.000409       2        1  25.405504
2     -0.025467 -0.000754       3        1  25.349377
0      0.004798 -0.016996       4        1  16.996827, topic_info=     Term         Freq        Total Category  logprob  loglift
85     백신  2216.000000  2216.000000  Default  30.0000  30.0000
101    생산   802.000000   802.000000  Default  29.0000  29.0000
134    으로  2421.000000  2421.000000  Default  28.0000  28.0000
493    국내   658.000000   658.000000  Default  27.0000  27.0000
676    모더   487.000000   487.000000  Default  26.0000  26.0000
..    ...          ...          ...      ...      ...      ...
215    회복    84.413248   577.255459   Topic4  -5.5781  -0.1504
45     다고    87.749884   694.101856   Topic4  -5.5393  -0.2960
197  플레이어    94.772455   879.425004   Topic4  -5.4623  -0.4556
776    방역    72.897948   427.709968   Topic4  -5.7248   0.0028
800    확진    71.345465   605.967965   Topic4  -5.7463  -0.3671

[364 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.263299   가능
0         2  0.338998   가능
0         3  0.253426   가능
0         4  0.148106   가능
1210      1  0.257114   감소
...     ...       ...  ...
2194      2  0.404029   휴무
1275      1  0.262682   흑자
1275      2  0.116748   흑자
1275      3  0.525364   흑자
1275      4  0.087561   흑자

[904 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 1])